In [6]:
! pip install --upgrade vocos encodec librosa safetensors

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import pprint
import IPython.display as ipd
import torch
import librosa

In [8]:
# load model
mars5, config_class = torch.hub.load('Camb-ai/mars5-tts', 'mars5_english', trust_repo=True)

Using cache found in /home/e5000862/.cache/torch/hub/Camb-ai_mars5-tts_master
/home/e5000862/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/e5000862/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/e5000862/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
# download example ref audio
! wget -O example.wav https://github.com/Camb-ai/mars5-tts/raw/master/docs/assets/example_ref.wav 

--2024-07-02 12:28:13--  https://github.com/Camb-ai/mars5-tts/raw/master/docs/assets/example_ref.wav
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Camb-ai/mars5-tts/master/docs/assets/example_ref.wav [following]
--2024-07-02 12:28:13--  https://raw.githubusercontent.com/Camb-ai/mars5-tts/master/docs/assets/example_ref.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137392 (134K) [audio/wav]
Saving to: ‘example.wav’

example.wav         100%[===================>] 134.17K  --.-KB/s    in 0.02s   

2024-07-02 12:28:13 (6.79 MB/s) - ‘example.wav’ saved [137392/137392]



In [3]:
! pip install librosa

Defaulting to user installation because normal site-packages is not writeable


In [10]:

wav, sr = librosa.load('./example.wav', 
                       sr=mars5.sr, mono=True)
wav = torch.from_numpy(wav)
ref_transcript = "We actually haven't managed to meet demand."
print("Reference audio:")
ipd.display(ipd.Audio(wav.numpy(), rate=mars5.sr))
print(f"Reference transcript: {ref_transcript}")

Reference audio:


Reference transcript: We actually haven't managed to meet demand.


In [11]:
deep_clone = True # set to False if you don't know prompt transcript or want fast inference.
# Below you can tune other inference settings, like top_k, temperature, top_p, etc...
cfg = config_class(deep_clone=deep_clone, rep_penalty_window=100,
                      top_k=100, temperature=0.7, freq_penalty=3)

ar_codes, wav_out = mars5.tts("The quick brown rat.", wav, 
          ref_transcript,
          cfg=cfg)

print('Synthesized output audio:')
ipd.Audio(wav_out.numpy(), rate=mars5.sr)

Note: using deep clone. Assuming input `c_phones` is concatenated prompt and output phones. Also assuming no padded indices in `c_codes`.
New x: torch.Size([1, 1671, 8]) | new x_known: torch.Size([1, 1671, 8]) . Base prompt: torch.Size([1, 215, 8]). New padding mask: torch.Size([1, 1671]) | m shape: torch.Size([1, 1671, 8])


: 

In [ ]:
pprint.pprint(config_class())

In [ ]:
ar_wav = mars5.vocode(ar_codes.cpu()[:, None])
ipd.Audio(ar_wav.numpy(), rate=mars5.sr)